<a href="https://colab.research.google.com/github/reidnersousa/IFB_Compiladores_2023/blob/main/Producao_Trabalho_Compiladores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from grammar import Grammar
from token_sequence import token_sequence
from predict import predict_algorithm

def create_ac_grammar()->Grammar:
    G = Grammar()

    G.add_terminal('float')
    G.add_terminal('int')
    G.add_terminal('id')

    G.add_terminal('assign')
    G.add_terminal('plus')
    G.add_terminal('minus')
    G.add_terminal('mul')
    G.add_terminal('div')

    G.add_terminal('inum')
    G.add_terminal('fnum')
    G.add_terminal('nomeVariavel')

    G.add_terminal('lbracket')   #[
    G.add_terminal('rbracket')   #]
    G.add_terminal('lbrace')     #{
    G.add_terminal('rbrace')     #}
    G.add_terminal('lparen')     #(  
    G.add_terminal('rparen')     #)

    
    G.add_terminal('semicolon')   #;
    G.add_terminal('comma')       #,
    G.add_terminal('maior')       #
    G.add_terminal('maior_igual')
    G.add_terminal('menor')
    G.add_terminal('menor_igual')
    G.add_terminal('igual')
    

    G.add_terminal('if')
    G.add_terminal('else')
    G.add_terminal('while')

    
    G.add_nonterminal('Programa')
    G.add_nonterminal('Declaracoes')
    G.add_nonterminal('Declaracao')
    G.add_nonterminal('Declaracao_Variaveis')
    G.add_nonterminal('Bloco')
    G.add_nonterminal('Tipo')
    G.add_nonterminal('Expressao_Aritmetica')
    G.add_nonterminal('Expressao_Aritmetica\'')
    G.add_nonterminal('Expressao_Comparativa')
    G.add_nonterminal('Comparacao')
    G.add_nonterminal('Atribuicao')

    G.add_nonterminal('Constante')
    G.add_nonterminal('Operacao')
    G.add_nonterminal('Identificador')
    G.add_nonterminal('Instrucao_condicional')
    G.add_nonterminal('Instrucao_print')
    G.add_nonterminal('Entao')
    
    ###########################################################################################
    #################################### 
    ##############################################################################################
    G.add_production('Programa', ['Declaracoes'])
    G.add_production('Declaracoes', ['Declaracao', 'Declaracoes'])
    G.add_production('Declaracoes', [])
    G.add_production('Declaracao', ['Declaracao_Variaveis', 'semicolon'])
    G.add_production('Declaracao', ['Bloco'])
    G.add_production('Declaracao', ['Expressao', 'semicolon'])
    G.add_production('Declaracao', ['Instrucao_Condicional'])
    G.add_production('Declaracao', ['Instrucao_Repeticao'])
    G.add_production('Instrucao_Repeitcao',['while','lparen','Expressao_Comparativa','rparen','lbrace','Bloco','rbrace'])
   

    G.add_production('Declaracao', ['Declaracao_Variaveis', 'Atribuicao', 'semicolon'])
    G.add_production('Atribuicao', ['Identificador', 'assign', 'Expressao'])

    G.add_production('Instrucao_Condicional', ['if', 'lparen', 'Expressao_Comparativa', 'rparen', 'Bloco', 'Entao'])
    G.add_production('Entao', ['else,''lbrace', 'Bloco', 'rbrace'])
    G.add_production('Entao', ['Instrucao_Condicional'])

    
    G.add_production('Declaracao_Variaveis', ['Tipo', 'Identificador', 'semicolon'])

    G.add_production('Bloco', ['lbrace', 'Declaracoes', 'Instrucoes', 'rbrace'])
    
    G.add_production('Instrucoes', ['Instrucao', 'Instrucoes'])
    G.add_production('Instrucoes', [])
    G.add_production('Instrucao', ['Declaracao'])
    G.add_production('Instrucao', ['Atribuicao', 'semicolon'])


    G.add_production('Expressao', ['Termo'])
    G.add_production('Expressao', ['Expresao','plus','Termo'])
    G.add_production('Expressao', ['Expresao','minus','Termo'])

    G.add_production('Termo',['Fator'])
    G.add_production('Termo',['Termo','mul','Fator'])
    G.add_production('Termo',['Termo','div','Fator'])
    
    G.add_production('Fator',['fnum'])
    G.add_production('Fator',['inum'])
    G.add_production('Fator',['lparen','Expressao','rparen'])
   
    G.add_production('Instrucao_print', ['print', 'lparen', 'Identificador', 'rparen', 'semicolon'])

    G.add_production('Expressao_Comparativa', ['Constante', 'Comparacao', 'Identificador'])
    G.add_production('Expressao_Comparativa', ['Identificador', 'Comparacao', 'Identificador'])
    
    G.add_production('Expressao_Comparativa', ['Constante', 'Comparacao', 'Identificador'])  ## nao é LL(K)


    G.add_production('Comparacao', ['maior'])
    G.add_production('Comparacao', ['maior_igual'])
    G.add_production('Comparacao', ['menor'])
    G.add_production('Comparacao', ['menor_igual'])
    G.add_production('Comparacao', ['igual'])

    G.add_production('Constante', ['inum'])
    G.add_production('Operacao', ['plus'])
    G.add_production('Operacao', ['minus'])

    G.add_production('Tipo', ['int'])
    G.add_production('Tipo', ['float'])

    G.add_production('Identificador', ['nomeVariavel'])
    G.add_production('Identificador', ['nomeVariavel', 'digitos'])

    return G




regex_table = {
    r'^f$': 'float',
    r'^i$': 'int',
    r'^p$': 'print',
    r'^[abcdeghjlmnoqrstuvwxyz]$': 'nomeVariavel',
    r'^=$': 'assignment',
    r'^\+$': 'plus',
    r'^\-$': 'minus',
    r'^\*$':'mul',
    r'^\\$':'div',
    r'^\%$':'rem',
    r'^==$':'igual',
    r'^>$' :'maior',
    r'^>=$':'maior_igual',
    r'^<=$':'menor_igual',
    r'^<$':'menor',
    r'^[0-9]+$': 'digitos',
    r'^[0-9]+\.[0-9]+$': 'fnum',
    r'^\[$': 'lbracket',
    r'^\]$': 'rbracket',
    r'^\($':'lparen',
    r'^\)$':'rparen',
    r'^\{$':'lbrace',
    r'^\}$':'rbrace',
    r'^;$': 'semicolon',
    r'^,$':'comma',
   

}

def lexical_analyser(filepath) -> str:
    with open(filepath,'r') as f:
        token_sequence = []
        tokens = []
        for line in f:
            tokens = tokens + line.split(' ')
        for t in tokens:
            found = False
            for regex,category in regex_table.items():
                if re.match(regex,t):
                    token_sequence.append(category)
                    found=True
            if not found:
                print('Lexical error: ',t)
                exit(0)
    token_sequence.append('$')
    return token_sequence

def Programa(ts:token_sequence,p:predict_algorithm)->None:
    if ts.peek() in p.predict(16):
        Declaracoes(ts,p)
        
        

def Declaracoes(ts:token_sequence , p:predict_algorithm)->None:
    if ts.peek() in p.predict(17):
        Declaracao (ts,p)
        Declaracoes(ts,p)
    elif ts.peek() in p.predict(18):
        return

def Declaracao(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict(19):
        Declaracao_Variaveis(ts,p)
        ts.match('semicolon')
    elif ts.peek()in p.predict(20):
        Declaracao_Funcoes(ts,p)
        ts.match('semicolon')
    elif ts.peek()in p.predict():
        Expressao(ts,p)

def Expressao(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict(21):
        Expressao_Arimetica(ts,p)
    elif ts.peek()in p.predict(00):
        Expressao_Logica(ts,p)
    


def Expressao_Aritmetica(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict(23):
       Expressao_Aritmetica(ts,p)
       Operacao(ts,p)
       Expressao_Aritmetica(ts,p)
    elif ts.peek()in p.predict(24):
       Constante(ts,p)

def Constante(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek()in p.predict():
      ts.match('inum')


def Operacao(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek()in p.predict():
      ts.match('plus')


def Declaracao_Variaveis(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict(25):
        
        Tipo(ts,p)
        Identificador(ts,p)
        ts.match('semicolon')

    elif ts.peek()in p.predict(26):
        Tipo(ts,p)
        Identificador(ts,p)
        ts.match('assign')
        Lista(ts,p)
        ts.match('semicolon')
  
        


def Lista(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict(28):
        ts.match('lbracket')
        Elementos(ts,p)
        ts.match('rbracket')


def Elementos(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict():
        Elementos(ts,p)
        ts.match('comma')
        Elementos(ts,p)
    elif ts.peek() in p.predict():
        return 

def Declaracao_Funcoes(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict():
        Tipo(ts,p)
        Identificador(ts,p)
        ts.match('rparen')
        Parametros(ts,p)
        ts.match('lparen')

def Parametros(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peel() in p.predict():
        Lista_parametro(ts,p)
    elif ts.peek() in p.predict():
        return 


def Lista_parametro(ts:token_sequence, p:predict_algorithm)->None:
    if ts.peek() in p.predict():
        Parametros(ts,p)
        ts.match('comma')
        Lista_parametro(ts,p)
    elif ts.peek() in p.predict():
        Parametros(ts,p)

if __name__ == '__main__':
    filepath = 'programa.ac'
    tokens = lexical_analyser(filepath)
    ts = token_sequence(tokens)
    G = create_ac_grammar()
    p_alg = predict_algorithm(G)
    Prog(ts,p_alg)  

In [ ]:
1+1+1+1


4

In [ ]:
a =[10]
print(len(a))
a=[10,10]
print(a)

1
[10, 10]
